In [13]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import RMSprop, Adam, Adagrad, SGD

import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterSampler

import os

batch_size = 32
num_classes = 10
epochs = 3
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [14]:
def get_cnn(conv_blocks_filters=[32, 64], cnn_dropout=0.25,
            conv_filter_size=(3, 3),
            pool_type='max', pool_size=(2, 2), activation='relu',
            clf_layers=[512, 128, 64], clf_dropout=0.5,
            optimizer='adam', learning_rate=0.001):
    """
    Function that compiles basic convolutional neural network
    Args:
        conv_blocks_filters: number of filters for each block (conv, conv, pool)
        cnn_dropout: dropout after the cnn block
        conv_filter_size: size of convolutional filters
        pool_type: type of pooling. Could be "max" or "average"
        pool_size: size of pooling
        activation: name of the activation function
        clf_layers: list with neuron number for each layer of the classifier neural network
        clf_dropout: dropout rate in the classifier(clf) nn
        optimizer: name of optimizer for the neural network
        learning_rate: learning rate for the optimizer

    Returns:
        compiled model
    """

    model = Sequential()

    for cnn_filters in conv_blocks_filters:
        model.add(Conv2D(cnn_filters, conv_filter_size,
                         padding='same'))
        model.add(Activation(activation))

        model.add(Conv2D(cnn_filters, conv_filter_size))
        model.add(Activation(activation))

        if pool_type == 'max':
            model.add(MaxPooling2D(pool_size=pool_size))
        elif pool_type == 'average':
            model.add(AveragePooling2D(pool_size=pool_size))
        else:
            raise ValueError(f"pool_type {pool_type} not recognized")
        model.add(Dropout(cnn_dropout))

    model.add(Flatten())

    for neuron_num in clf_layers:
        model.add(Dense(neuron_num))
        model.add(Activation(activation))
        model.add(Dropout(clf_dropout))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # Selecting optimizer
    if optimizer == 'rmsprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(lr=learning_rate)
    elif optimizer == 'adamgrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(lr=learning_rate)
    else:
        raise ValueError(f"optimizer {optimizer} not recognized")

    # Let's train the model using optimizer
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [16]:
hyperspace_grid = {'conv_blocks_filters':[ [32, 64], [16, 32]],
                   'cnn_dropout': [0.15, 0.3],
                   'conv_filter_size': [(3,3), (2,2)],
                   'pool_type': ['max', 'average'],
                   'pool_size': [(3,3), (2,2)],
                   'activation': ['relu', 'sigmoid'],
                   'clf_layers': [ [512], [512, 128, 64]],
                   'clf_dropout': [0.25, 0.5],
                   'optimizer': ['adam', 'sgd'],
                   'learning_rate': [0.0001, 0.001, 0.01]
                  }

# We are taking 10 random configurations from this grid. NOT ALL!
n_configurations = 10
configurations_list = list(ParameterSampler(hyperspace_grid,
                                            n_iter=n_configurations))

configurations_loss = []
configurations_accuracy = []

for configuration in configurations_list:
    print(configuration)
    model = get_cnn(**configuration)
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

    # Save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    configurations_loss.append(scores[0])
    configurations_accuracy.append(scores[1])
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    
results = pd.DataFrame({'loss':configurations_loss,
                        'accuracy': configurations_accuracy,
                        'params': str(configuration)})
results

{'pool_type': 'max', 'pool_size': (3, 3), 'optimizer': 'sgd', 'learning_rate': 0.001, 'conv_filter_size': (3, 3), 'conv_blocks_filters': [16, 32], 'cnn_dropout': 0.3, 'clf_layers': [512, 64], 'clf_dropout': 0.25, 'activation': 'sigmoid'}
Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 120s 2ms/step - loss: 2.3938 - acc: 0.1016 - val_loss: 2.3029 - val_acc: 0.1000
Epoch 2/3
50000/50000 [==============================] - 119s 2ms/step - loss: 2.3678 - acc: 0.0986 - val_loss: 2.3027 - val_acc: 0.1000
Epoch 3/3
50000/50000 [==============================] - 118s 2ms/step - loss: 2.3590 - acc: 0.0992 - val_loss: 2.3029 - val_acc: 0.1000
Saved trained model at C:\Users\User\Downloads\saved_models\keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 9s 876us/step
Test loss: 2.30288709564209
Test accuracy: 0.1
{'pool_type': 'average', 'pool_size': (3, 3), 'optimizer': 'sgd', 'learning_rate': 0.0001, 'conv_filt

KeyboardInterrupt: 